In [1]:
import os
import ansys.speos.script as script
from ansys.speos.core import Speos
import ansys.speos.workflow as workflow

speos = Speos(host="localhost", port=50052)

In [2]:
project = script.Project(speos=speos)
opt_material_1 = project.create_optical_property("optical-polished")
opt_material_1.set_volume_optic(index=1.5, absorption=0, constringence=60).set_surface_opticalpolished().commit()

opt_material_2 = project.create_optical_property("black")
opt_material_2.set_volume_none().set_surface_mirror(reflectance=0).commit()


In [3]:
root_part = project.create_root_part().commit()

source_body = root_part.create_body(name="source_surface").commit()
source_body.create_face(name="face1").set_vertices(
    [-5, -11.228, 5,
     5, -11.228, 5,
     5, -11.228, -5,
     -5, -11.228, -5
     ]).set_facets(
    [1, 2, 0,
     2, 3, 0
     ]).set_normals(
    [0.0, 1.0, 0.0,
     0.0, 1.0, 0.0,
     0.0, 1.0, 0.0,
     0.0, 1.0, 0.0
     ]).commit()

prism_body = root_part.create_body(name="prism_body").commit()
for face_id in range(6):
    face = prism_body.create_face(name="face"+str(face_id)).set_facets(
        [1, 2, 0,
         2, 3, 0
     ])
    if face_id == 0:
        face.set_vertices(
            [-5, 0.0, 5,
             -5, 0.0, -5,
             5, 0.0, -5,
             5, 0.0, 5
             ]).set_normals(
            [-0.0, -1.0, -0.0,
             -0.0, -1.0, -0.0,
             -0.0, -1.0, -0.0,
             -0.0, -1.0, -0.0
             ]).commit()
    elif face_id == 1:
        face.set_vertices(
            [5, 0.0, -5,
             5, 10, -5,
             5, 10, 5,
             5, 0.0, 5,
            ]).set_normals(
            [1.0, -0.0, 0.0,
             1.0, -0.0, 0.0,
             1.0, -0.0, 0.0,
             1.0, -0.0, 0.0
             ]).commit()
    elif face_id == 2:
        face.set_vertices(
            [-5, 10, 5,
             5, 10, 5,
             5, 10, -5,
             -5, 10, -5,
             ]).set_normals(
            [0.0, 1.0, 0.0,
             0.0, 1.0, 0.0,
             0.0, 1.0, 0.0,
             0.0, 1.0, 0.0
             ]).commit()
    elif face_id == 3:
        face.set_vertices(
            [-5, 0.0, -5,
             -5, 10, -5,
             5, 10, -5,
             5, 0.0, -5,
             ]).set_normals(
            [0.0, 0.0, -1.0,
             0.0, 0.0, -1.0,
             0.0, 0.0, -1.0,
             0.0, 0.0, -1.0
             ]).commit()
    elif face_id == 4:
        face.set_vertices(
            [-5,0.0,5,
             -5,10,5,
             -5,10,-5,
             -5,0.0,-5,
             ]).set_normals(
            [-1.0, 0.0, 0.0,
             -1.0, 0.0, 0.0,
             -1.0, 0.0, 0.0,
             -1.0, 0.0, 0.0
             ]).commit()
    elif face_id == 5:
        face.set_vertices(
            [5,0.0,5,
             5,10,5,
             -5,10,5,
             -5,0.0,5,
             ]).set_normals(
            [0.0, 0.0, 1.0,
             0.0, 0.0, 1.0,
             0.0, 0.0, 1.0,
             0.0, 0.0, 1.0
             ]).commit()

opt_material_1.set_geometries(
    geometries=[script.GeoRef.from_native_link(geopath="prism_body")]
).commit()

opt_material_2.set_geometries(
    geometries=[script.GeoRef.from_native_link(geopath="source_surface")]
).commit()

In [4]:
source_1 = project.create_source(name="surface_source")
source_1.set_surface().set_spectrum().set_blackbody()
source_1.set_surface().set_exitance_constant(geometries=[(script.GeoRef.from_native_link(geopath="source_surface/face1"), False)])
source_1.commit()

In [5]:
sensor_1 = project.create_sensor(name="irradiance_sensor")
sensor_1.set_irradiance().set_type_spectral()
sensor_1.set_irradiance().set_axis_system(
    [0, 20, 0,
     1, 0, 0,
     0, 0, 1,
     0, 1, 0
     ])
sensor_1.set_irradiance().set_illuminance_type_planar(integration_direction=[0, -1, 0])
sensor_1.commit()

In [6]:
workflow.preview(project)

Widget(value='<iframe src="http://localhost:56126/index.html?ui=P_0x2a12f199290_0&reconnect=auto" class="pyvis…

In [7]:
print(project)

ansys.api.speos.scene.v2.Scene
{
    "part_guid": "94281f17-1a09-4c6e-92ef-8ca56d753cfa",
    "sources": [
        {
            "name": "surface_source",
            "metadata": {
                "UniqueId": "221022a6-ee08-4a01-9387-4e74111c28f7"
            },
            "source_guid": "b2f576c0-42f0-468e-ac02-8d92c10a0b98",
            "surface_properties": {
                "exitance_constant_properties": {
                    "geo_paths": [
                        {
                            "geo_path": "source_surface/face1",
                            "reverse_normal": false
                        }
                    ]
                },
                "intensity_properties": {}
            },
            "description": ""
        }
    ],
    "sensors": [
        {
            "name": "irradiance_sensor",
            "metadata": {
                "UniqueId": "892238bb-a0b5-4dc6-8ce1-d0343e90495d"
            },
            "sensor_guid": "6a0afe6d-0a8e-46e2-bfdc-37e66a6

In [8]:
simu = project.create_simulation(name="direct.1.pyspeos")
simu.set_direct()
simu.set_sensor_paths(["irradiance_sensor"])
simu.set_source_paths(["surface_source"])
simu.set_direct().set_stop_condition_rays_number(value=20000000)
simu.commit()

In [9]:
print(project)


ansys.api.speos.scene.v2.Scene
{
    "part_guid": "94281f17-1a09-4c6e-92ef-8ca56d753cfa",
    "sources": [
        {
            "name": "surface_source",
            "metadata": {
                "UniqueId": "221022a6-ee08-4a01-9387-4e74111c28f7"
            },
            "source_guid": "b2f576c0-42f0-468e-ac02-8d92c10a0b98",
            "surface_properties": {
                "exitance_constant_properties": {
                    "geo_paths": [
                        {
                            "geo_path": "source_surface/face1",
                            "reverse_normal": false
                        }
                    ]
                },
                "intensity_properties": {}
            },
            "description": ""
        }
    ],
    "sensors": [
        {
            "name": "irradiance_sensor",
            "metadata": {
                "UniqueId": "892238bb-a0b5-4dc6-8ce1-d0343e90495d"
            },
            "sensor_guid": "6a0afe6d-0a8e-46e2-bfdc-37e66a6

In [10]:
simu.compute_CPU()

[path: "C:\\Users\\plu\\AppData\\Local\\Temp\\jobs\\f1b01ae2-ac3b-4fc9-9dce-1846674b79f3\\irradiance_sensor.xmp"
, path: "C:\\Users\\plu\\AppData\\Local\\Temp\\jobs\\f1b01ae2-ac3b-4fc9-9dce-1846674b79f3\\direct.1.pyspeos.html"
]

In [12]:
sim2 = project.create_simulation("interactive")
sim2.set_interactive()
sim2.set_source_paths(["surface_source"])
sim2.set_sensor_paths(["irradiance_sensor"])
sim2.set_interactive().set_light_expert(True)
sim2.commit()
print(sim2)
result = sim2.compute_CPU()

local: ansys.api.speos.job.v2.Job
{
    "scene_guid": "bed297f6-24fc-42b0-963e-a4d50696d158",
    "simulation_path": "interactive",
    "interactive_simulation_properties": {
        "light_expert": true,
        "rays_number_per_sources": [],
        "impact_report": false
    },
    "name": "interactive.Job",
    "description": "",
    "metadata": {},
    "job_type": "CPU"
}ansys.api.speos.scene.v2.Scene.SimulationInstance
{
    "name": "interactive",
    "metadata": {
        "UniqueId": "8882defb-a48b-4a2d-9f83-75cdb40b783d"
    },
    "simulation_guid": "892be04e-e5fc-4c1e-92b1-25da6b6cf0ac",
    "sensor_paths": [
        "irradiance_sensor"
    ],
    "source_paths": [
        "surface_source"
    ],
    "description": ""
}
ansys.api.speos.simulation.v1.templates.SimulationTemplate
{
    "name": "interactive",
    "interactive_simulation_template": {
        "geom_distance_tolerance": 0.01,
        "max_impact": 100,
        "weight": {
            "minimum_energy_percentage": 0.

In [16]:
print(result[0].path)

from ansys.speos.script.lxp import LightPathFinder
lxp= script.LightPathFinder(speos,result[0].path)
print(lxp.sensor_contributions)
lxp.preview(project=project)


C:\Users\plu\AppData\Local\Temp\jobs\097e8829-1940-4b69-bbaf-d33601731fa5\interactive.lpf
None


Widget(value='<iframe src="http://localhost:56126/index.html?ui=P_0x2a13ce96890_1&reconnect=auto" class="pyvis…

In [17]:
lxp.filter_error_rays()
lxp.preview(project=project,ray_filter=True)

Widget(value='<iframe src="http://localhost:56126/index.html?ui=P_0x2a147706c10_2&reconnect=auto" class="pyvis…

In [20]:
print(project)

ansys.api.speos.scene.v2.Scene
{
    "part_guid": "94281f17-1a09-4c6e-92ef-8ca56d753cfa",
    "sources": [
        {
            "name": "surface_source",
            "metadata": {
                "UniqueId": "221022a6-ee08-4a01-9387-4e74111c28f7"
            },
            "source_guid": "b2f576c0-42f0-468e-ac02-8d92c10a0b98",
            "surface_properties": {
                "exitance_constant_properties": {
                    "geo_paths": [
                        {
                            "geo_path": "source_surface/face1",
                            "reverse_normal": false
                        }
                    ]
                },
                "intensity_properties": {}
            },
            "description": ""
        }
    ],
    "sensors": [
        {
            "name": "irradiance_sensor",
            "metadata": {
                "UniqueId": "892238bb-a0b5-4dc6-8ce1-d0343e90495d"
            },
            "sensor_guid": "6a0afe6d-0a8e-46e2-bfdc-37e66a6

In [21]:
for ray in lxp.rays:
    print(ray.body_ids)

[874039043]
[874039043]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043, 1857398667, 1857398667]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043, 1857398667, 1857398667]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043, 1857398667, 1857398667]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043]
[874039043]
[874039043, 1857398667, 1857398667]
[874039043]
[874039043]
[874039043, 1857398667, 1857398667, 1857398667]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043]
[874039043, 1857

In [29]:
project.preview({"opacity": 1})

Widget(value='<iframe src="http://localhost:56126/index.html?ui=P_0x2a147358dd0_10&reconnect=auto" class="pyvi…